#### Pseudo Code

- [X] Parse "Curve Header Data" tab from curve PSD
- [X] Create dictionary/dataframe?
- [X] Create Root element and namespace ('SKBData')
- [X] Add Child Element and attributes('CurveFamily')
- [ ] Convert and insert header_dict as grandchildren of CurveFamily Element
- [ ] Create (w/ modified values) and Insert 'pumpCurveCollection' element for each tab in NBS Curve PSD
- [ ] Create and Insert 'Impeller' Element as children to 'pumpCurveCollection' element.

#### File Setup

In [809]:
from lxml import etree as ET
import os
import pandas as pd
import re

In [810]:
# Use these files to grab info off of original NBS curve export.
fileDir = r"C:\Ogma DB Files"
filename = "NBS_Std_Curve_Export 2022-12-7.xml"
filepath = os.path.join(fileDir, filename)

import xml.etree.ElementTree as ET2

src_tree = ET2.parse(filepath)
src_root = src_tree.getroot()


In [811]:
psd_path = r"C:\Users\104092\OneDrive - Grundfos\Documents\10-19 Projects\12 NBS Curve PSD Separation\12.02 Output Files"
psd_file = r"GXS Curve_Conexus_V2 - std models.xlsx"
psd_filepath = os.path.join(psd_path, psd_file)

#### Functions 

In [812]:
def add_namespace(elem_tag, xsi_namespace):
    XHTML_NAMESPACE = xsi_namespace
    XHTML = "{%s}" % XHTML_NAMESPACE
    NSMAP = {'xsi' : XHTML_NAMESPACE} # the default namespace (no prefix)

    return ET.Element(elem_tag, nsmap=NSMAP) # lxml only!

In [813]:
def add_elem_from_dict(parent_elem, elem_dict):
    """Takes elements inside elem_dict and adds as elements to parent_elem"""
    for key, value in elem_dict.items():
        elem = ET.SubElement(parent_elem, key)
        elem.text = str(value)

In [814]:
def copy_values_from_source_xml(updates_list, new_curve_num):

    # Extract diameter (in mm) from curve name
    res = re.search("-(\d+)_Std", new_curve_num)
    curve_trim_size_mm = int(res.group(1))
    print(f'curve trim size in mm: {curve_trim_size_mm} mm')

    updates_dict = {}

    # List of all <pumpCurveCollection> nodes  
    for elem in src_root.findall(".//pumpCurveCollection"):       

        # Get reference diameter from original nbs curve xml to compare to trim in new curve xml
        original_curve = elem.find('curveNumber').text  # 012-070-2P

        if original_curve[:-4] in new_curve_num: # original_Curve: 012-070-2P, new_curve: 012-070-2P-109_Std
            print(f'original_Curve: {original_curve[:-4]}, new_curve: {new_curve_num}')

            # Get Impeller node to dig into
            for impeller_node in elem.findall('.//Impeller'):

                x = impeller_node.find('./diameter')              
                diameters_int = round(float(x.text))

                if curve_trim_size_mm == diameters_int:
                    print(f"diameters_int: {diameters_int}, type: {type(diameters_int)}, curve_trim_mm: {curve_trim_size_mm}, type: {type(curve_trim_size_mm)}")
                        
                    for item in updates_list:
                        updates_dict.update({ item: impeller_node.find('./'+item).text })
         
            print(f"updates_dict: {updates_dict}")
            return(updates_dict)


In [815]:
def create_impeller_dict(new_curve_number):

	# # Extract diameter (in mm) from curve name
	# res = re.search("-(\d+)_Std", curve_number)
	# curve_trim_size_mm = int(res.group(1))
	# curve_trim_size_in = curve_trim_size_mm/25.4

    # Need to get these from nbs curve export 
	updates_list = [
		'diameter',
		'flowBEPFixed',
		'etaBEPFixed',
		'motorRatingMin',
		'motorRatingMax'
		]
		
	append_dict = copy_values_from_source_xml(updates_list, new_curve_number)
	# print(f"append_dict: {append_dict}")
	
	impeller_dict = {
	# 'diameter': curve_trim_size_in,
	# 'diameter': curve_trim_size_mm,
	'diameterHubSide':'0.0',
	'weight':'0.0',
	'surgeFlow':'0.0',
	'flowStartEta':'0.0',
	'flowStartHead':'0.0',
	'flowStartNPSH':'0.0',
	'flowStartNPSH0Percent':'0.0',
	'flowStartNPSHIncipient':'0.0',
	'flowStopNPSH':'0.0',
	'flowStopNPSH0Percent':'0.0',
	'flowStopNPSHIncipient':'0.0',
	'flowStartSubmergence':'0.0',
	'flowStartPower':'0.0',
	'powerShutoffFixedEnabled':'false',
	'powerShutoffFixed':'0.0',
	'bepFixedEnabled':'true',
	'solveVariantMin':'0.0',
	'solveVariantMax':'0.0',
	'minimumVolumetricEfficiency':'0.0',
	'minimumVolumetricEfficiencyRated':'0.0',
	'maximumDifferentialPressure':'0.0',
	'stopFlow':'0.0'
	}
	
	append_dict.update(impeller_dict)
	print(f"final dict to be added: {append_dict}")
	# Python code to merge dict using update() method
	return(append_dict)

In [816]:
def create_pump_curve_dict(row) -> dict:
    """Returns dictionary of updated attributes to be converted to elements """
    pumpCurve_dict = {
        'curveNumber': row['Curve number'],
        'speedRef': row['Speed, data'],
        'polesRef': row['Poles'],
        'hzRef': row['Hz'],
        'mcsfMinRef': row['MCSF @ min impeller diameter'],
        'mcsfMaxRef': row['MCSF @ max impeller diameter'],
        'eyeCount': row['Number of impeller eyes'],
        'speedCurveNominal': row['Speed, nominal'],
        'speedCurveMin': row['Speed, Min'],
        'speedCurveMax': row['Speed, max'],
        'diaImpInc': row['Diameter increment'],
        'speedVariableCurveMin': row['Variable speed min limit'],
        'optionalCurveType': 'Power',
        'flowStartHeadEnabled': 'false',
        'flowStartEtaEnabled': 'false',
        'flowStartPowerEnabled': 'false',
        'speedVariableCurveMax':'0',
		'flowStartNPSHEnabled':'false',
		'flowStopNPSHEnabled':'false',
		'flowStartSubmergenceEnabled':'false',
		'extendNpshToMcsfMin':'false',
		'catalogTrimsSelectionMode':'0',
		'styleCurveBelowStart':'none',
		'flowExponentTrim':'1.0',
		'headExponentTrim':'2.0',
		'npshExponentTrim':'0.0',
		'etaExponentTrim':'0.0',
		'powerDriverFixed':'0.0',
		'quantityMotors':'1',
		'serviceFactorDriverFixed':'1.0',
		'serviceFactorDriverFixedUsed':'false',
		'flowExponentSpeed':'1.0',
		'headExponentSpeed':'2.0',
		'etaExponentSpeedReduced':'0.0',
		'etaExponentSpeedIncreased':'0.0',
		'npshExponentSpeedReduced':'2.0',
		'npshExponentSpeedIncreased':'2.0',
		'submergenceExponentSpeedReduced':'2.0',
		'submergenceExponentSpeedIncreased':'2.0',
		'hideEfficiencyInSelector':'false',
		'speedOfSoundRef':'331.6583',
		'speedOfSoundExpFlow':'1.0',
		'speedOfSoundExpHead':'2.0',
		'speedOfSoundExpEta':'0.0',
		'speedOfSoundExpEtaTotal':'0.0',
		'temperatureGasInletSkb':'20.0',
		'pressureGasInletSkb':'1.01325',
		'relativeHumidityGasSkb':'50.0',
		'diaRotatingElement':'0.0',
		'solveVariantDisplayStrategy':'2',
		'flowStopPercentBEP':'0.0',
		'headMarginFixedDia':'0.0',
		'headMarginFixedDiaPercentage':'0.0',
		'submergenceVortexMin':'0.0',
		'submergenceStartupMin':'0.0',
		'thrustFactor':'0.0',
		'thrustFactorBalanced':'0.0',
		'displayBothDiameters':'false',
		'isoEfficiencyValues':'56:62:65:68',
		'moiFirstStage':'0.0',
		'moiAdditionalStage':'0.0',
		'moiPumpCoupling':'0.0',
		'flowMaxAllowedMinRef':'0.0',
		'flowMaxAllowedMaxRef':'0.0',
		'loadRadialRef':'0.0'
		}

    return pumpCurve_dict

In [817]:
def metric_to_us(input_value, parameter_type:str):
    if parameter_type == 'flow':
        return input_value * 4.40286764029913
    elif parameter_type == 'distance':
        return input_value * 3.28083989501312
    elif parameter_type == 'power':
        return input_value * 1.3410218586563
    else:
        print("non valid parameter type entered")

In [818]:
def add_curve_data_points(parent_elem, curve_number, curve_type):
    
    curve_data_df = pd.read_excel(psd_filepath,sheet_name=curve_number, header=7, skiprows=[8], usecols="D,E,L,S", nrows=50)
    curve_data_df = curve_data_df.dropna()
    
    for index, row in curve_data_df.iterrows():
        datapoint_elem = ET.SubElement(parent_elem, "DataPoint", disabled="false")
        
        if curve_type == 'Power':
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'power'),
                'x': row['Flow'],
                'y': row[curve_type],
                'isOnCurve':'false',
                'division':'false',
                'useCubicSplines':'false',
                'slopeEnabled':'false'
            }

        elif (curve_type == 'Head'):
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'distance'),
                'x': row['Flow'],
                'y': row[curve_type],
                'isOnCurve':'false',
                'division':'false',
                'useCubicSplines':'true',
                'slopeEnabled':'false'
            }

        elif (curve_type == 'NPSH'):
            datapoint_dict = {
                # 'x': metric_to_us(row['Flow'], "flow"),
                # 'y': metric_to_us(row[curve_type], 'distance'),
                'x': row['Flow'],
                'y': row[curve_type],
                'isOnCurve':'false',
                'division':'false',
                'useCubicSplines':'false',
                'slopeEnabled':'false'
            }


        else:
            print(f'curve_type not allowed: {curve_type}')
            
        add_elem_from_dict(datapoint_elem, datapoint_dict)

In [819]:
def add_curve(parent_elem, curve_type:str, curve_number):
    curve_elem = ET.SubElement(parent_elem, 'Curve', type=curve_type)

    # Add Curve Data Points to Curve Element
    add_curve_data_points(curve_elem, curve_number, curve_type)

#### Create Root Element (and namespace)

In [820]:
root_ns = "http://www.w3.org/2001/XMLSchema-instance"
root = add_namespace('SKBData', root_ns)

#### Create CurveFamily Element as subelem to root

In [821]:
curveFamily_elem = ET.SubElement(root, "CurveFamily", selectorVersion="8.0.0", skbVersion="22.2.0.220418.623")

curve_family_name = "NBS_Fixed_Trim"

header_dict = {
        'name': curve_family_name,
        'impellerTyp':'radialFlow',
        'svDataType':'impellerDiamter',
        'interpDataType':'impellerDiamter',
        'compressorConditionsInputTypeSkb':'speedOfSound',
        'flowTypeSkb':'volumetricFlow',
        'headTypeSkb':'head',
        'headMarginForFixedDiameter':'value',
        'submergenceMethod':'fixedValue',
        'errorFitMax':'1.5',
        'pumpType':'0',
        'interpQty':'4',
        'efficiencyPowerDataType':'pump'}

# Add header data as sub-elements to CurveFamily Tag       
# for key, value in header_dict.items():
#     elem = ET.SubElement(curveFamily_elem, key)
#     elem.text = value

add_elem_from_dict(curveFamily_elem, header_dict)

#### Inserting UnitofMeasureSettings as subelement to CurveFamily

In [822]:
# unitOfMeasureSettings
with open('unitofmeasuresettings-boilerplate.txt','r') as file:
	unit_of_measure_settings = file.read()
	
uom_element = ET.fromstring(unit_of_measure_settings)
curveFamily_elem.append(uom_element)

#### Add pumpCurveCollection for each curve tab

In [823]:
curve_header_data = pd.read_excel(psd_filepath,sheet_name="Curve Header Data", header=8, skiprows=[9])

# Need to set appropriate values before adding to XML tree. Will need to iterate through curve_header_data to update values.
for index, row in curve_header_data.iterrows():
    
    # <pumpCurveCollection xsi:type="CentrifugalPumpCurveCollection"> This is the parent of each pump curve"
    qname = ET.QName(root_ns,"type")
    pumpCurveCollection_elem = ET.SubElement(curveFamily_elem, 'pumpCurveCollection', {qname: "CentrifugalPumpCurveCollection"})

    # Creates pump curve elements
    curve_dict = create_pump_curve_dict(row)
    add_elem_from_dict(pumpCurveCollection_elem, curve_dict)

    # Add Impeller Elements
    impeller_elem = ET.SubElement(pumpCurveCollection_elem, 'Impeller')
    impeller_dict = create_impeller_dict(row['Curve number'])
    add_elem_from_dict(impeller_elem, impeller_dict)

    # Add Curve Element
    add_curve(impeller_elem, "Head", row['Curve number'])
    add_curve(impeller_elem, "Power", row['Curve number'])
    add_curve(impeller_elem, "NPSH", row['Curve number'])    

curve trim size in mm: 109 mm
original_Curve: 012-070-2P, new_curve: 012-070-2P-109_Std
diameters_int: 109, type: <class 'int'>, curve_trim_mm: 109, type: <class 'int'>
updates_dict: {'diameter': '108.966', 'flowBEPFixed': '17.85', 'etaBEPFixed': '50.22633461', 'motorRatingMin': '2.2371', 'motorRatingMax': '11.1855'}
final dict to be added: {'diameter': '108.966', 'flowBEPFixed': '17.85', 'etaBEPFixed': '50.22633461', 'motorRatingMin': '2.2371', 'motorRatingMax': '11.1855', 'diameterHubSide': '0.0', 'weight': '0.0', 'surgeFlow': '0.0', 'flowStartEta': '0.0', 'flowStartHead': '0.0', 'flowStartNPSH': '0.0', 'flowStartNPSH0Percent': '0.0', 'flowStartNPSHIncipient': '0.0', 'flowStopNPSH': '0.0', 'flowStopNPSH0Percent': '0.0', 'flowStopNPSHIncipient': '0.0', 'flowStartSubmergence': '0.0', 'flowStartPower': '0.0', 'powerShutoffFixedEnabled': 'false', 'powerShutoffFixed': '0.0', 'bepFixedEnabled': 'true', 'solveVariantMin': '0.0', 'solveVariantMax': '0.0', 'minimumVolumetricEfficiency': '0.0'

#### Write to file

In [824]:
et = ET.ElementTree(root)
et.write('nbs_fixed_trim_all_curves.xml', pretty_print=True)